# Código para la automatización de la extracción de datos. 
#### Alumna: Lucía Vítores López

## ÍNDICE:
1. [Importaciones](#1)



2. [Desarrollo código](#2)

    2.1. [Definición de funciones](#2.1.)
    
    2.2. [Importación de ficheros Excel](#2.2)
    
    2.3. [Definición de algunas variables](#2.3)
    
    2.4. [Diagnóstico](#2.3.1)
    
    2.5. [NHC](#2.3.2)
    
    2.6. [Biopsia](#2.3.3)
    
    2.7. [Biopsia sólida](#2.3.4)
    
    2.8. [Fecha](#2.3.5)
    
    2.9. [Definimos el resto de las variables](#2.4)
    
    2.10. [Ensayos clínicos y tratamientos disponibles](#2.4.1)
    
    2.11. [Número de chip y de paciente](#2.4.2)
    
    2.12. [Mutaciones y derivados](#2.4.3)
    
    2.13. [Mutaciones totales](#2.4.3.1)
    
    2.14. [Genes patogénicos](#2.4.3.2)
    
    2.15. [Variables de interés](#2.5)
    
    
    
3. [Creación de DataFrame](#3)


4. [Exportación](#4)

## 1. Importaciones. <a id = "1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

Hacemos las importaciones necesarias para que funcione el código.

Cada una de las bibliotecas estará correctamente explicada en la memoria disponible en GitHub. 

In [1]:
import fitz 
import re
import pandas as pd
import os
import numpy as np

## 2. Desarrollo código. <a id = "2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

## 2.1. Definición de funciones. <a id = "2.1."></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

Para la función `LeerFicherosPDF` se crea una lista vacía llamada **ficheros** y otra llamada **subcarpetas** que almacena las subcarpetas encontradas dentro de la ruta determinada por **os.walk(ruta)**. 

Se ordenan con **sorted()** usando el argumento **key** para hacerlo de forma numérica, sin tener en cuenta los valores no numéricos del nombre de cada carpeta. 


Se itera sobre las subcarpetas ordenadas y se lee cada archivo PDF dentro de ellas, agregándolos a la lista ficheros junto con su ruta completa.

La función devuelve la lista ficheros que contiene las rutas de todos los archivos PDF encontrados en las subcarpetas ordenadas.

In [2]:
'''def LeerFicherosPDF(ruta):
    ficheros = []

    for raiz, directorios, archivos in os.walk(ruta):
        for archivo in archivos:
            if archivo.endswith('.pdf'):
                ficheros.append(os.path.join(raiz,archivo))

    return ficheros
'''

In [107]:
def LeerFicherosPDF(ruta):
    ficheros = []
    subcarpetas = []

    for raiz, directorios, archivos in os.walk(ruta):
        # Obtener las subcarpetas y ordenarlas por nombre de forma numérica
        subcarpetas = sorted(directorios, key=lambda x: int(re.sub('\D', '', x)))

        for subcarpeta in subcarpetas:
            subcarpeta_ruta = os.path.join(raiz, subcarpeta)
            for archivo in os.listdir(subcarpeta_ruta):
                if archivo.endswith('.pdf'):
                    ficheros.append(os.path.join(subcarpeta_ruta, archivo))

    return ficheros


La función `LeerDocumento` lee el contenido de un documento usando la libreria **PyMuPDF** (fitz). 

Con **fitz.open()** abre el documento denominado **nombreFichero** e inicializa la variable **text** como cadena vacía.

El bucle for nos permite recorrer cada página del documento y obtener el texto que lo forma. 

Finalmente, la función devuelve el texto dividido con el separador \n.

In [3]:
def LeerDocumento(nombreFichero):
    with fitz.open(nombreFichero) as doc:
        text=""
        for page in doc:
            text = text + page.get_text()
    return text.split('\n')

La función `BuscarValor` nos permite buscar una palabra determinada en el texto de cada fichero. 

Busca la palabra definida en la cadena **textoBuscar** en la lista de cadenas **lines**, para ello usamos la biblioteca **re**. Se recorren las posiciones en busca de los valores definidos en **textoBuscar** y en caso de encontrar, la nueva variable **valores** almacenará un valor 1, en caso de que no esté almacena un 0. 

Con el bucle for recorremos cada uno de los elementos almacenados en **valores** y se usa la técnica slicing [:] para extraer la subcadena que hay tras la última vez que aparece la palabra, almacenando los resultados en la nueva variable **Encontrados**. Strip permite eliminar los espacios en blanco.

In [4]:
def BuscarValor(textoBuscar, lines):
    Encontrados = []

    valores = [1 if re.search(textoBuscar, line) else 0 for line in lines]
    valores = [i for i, s in enumerate(valores)  if s==1 in valores]

    posiciones = len(textoBuscar)
    
    for i in valores:
        Encontrados.append(lines[i][lines[i].rfind(textoBuscar)+posiciones:].strip())

    return Encontrados

Esta función `GenerarImagen` es un extra que sirve para crear imágenes partiendo de los documentos. De cada hoja del fichero PDF se obtiene una imagen con su contenido.

En caso de querer usarla solo hay que descomentar (eliminar ''')

In [12]:
''' def GenerarImagen(ruta, fichero):
    i = 0   
    zoom = 4
    mat = fitz.Matrix(zoom, zoom)
    with fitz.open(os.path.join(ruta,fichero)) as doc:
        for page in doc:
            i+=1
            val = f"image_{i+1}.png"
            pix = page.get_pixmap(matrix=mat)
            pix.save(os.path.join(ruta,fichero + "_" + val)) '''


' def GenerarImagen(ruta, fichero):\n    i = 0   \n    zoom = 4\n    mat = fitz.Matrix(zoom, zoom)\n    with fitz.open(os.path.join(ruta,fichero)) as doc:\n        for page in doc:\n            i+=1\n            val = f"image_{i+1}.png"\n            pix = page.get_pixmap(matrix=mat)\n            pix.save(os.path.join(ruta,fichero + "_" + val)) '

## 2.2. Importación de ficheros Excel. <a id = "2.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [5]:
# Importamos el fichero que almacena los nombres del diagnóstico y su número correspondiente.
# La ruta corresponde al lugar donde se encuentra el fichero Diagnostico.xlsx
diagnostico = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Datos/Diagnostico.xlsx")
#print(diagnostico)

#Creo un diccionario con los valores del fichero importado. 
diagnosticos_dic = dict(zip(diagnostico["DIAGNÓSTICO"], diagnostico["NÚMERO DIAGNÓSTICO"]))

# En cada iteración, almacena el diagnóstico y si valos correspondiente, almacenándolo en valor, lo que nos permite imprimir el diccionario en filas separadas. 
for diagnostico in diagnosticos_dic:
    valor = diagnosticos_dic.get(diagnostico)
    print(diagnostico, valor)


Carcinoma del tracto biliar 1.0
Colangiocarcinoma 1.1
Carcinoma de la vesícula biliar 1.2
Cáncer de vejiga 2.0
Carcinoma vesicular uroterial 2.1
Cáncer de mama 3.0
Triple negativo de mama 3.1
Neoplasia del sistema nervioso central 4.0
Glioma 4.1
Cáncer cervical 5.0
Condrosarcoma 6.0
Cáncer colorrectal 7.0
Carcinoma de células escamosas cutáneas 8.0
Carcinoma endometrial 9.0
 Cáncer esofágico 10.0
Cáncer gástrico 11.0
Tumor del estroma gastrointestinal 11.1
Cáncer de cabeza y cuello 12.0
Cáncer de riñón 13.0
Cáncer de hígado 14.0
Carcinoma hepatocelular 14.1
Cáncer de pulmón/pulmonar 15.0
Carcinoma pulmonar no microcítico 15.1
Carcinoma pulmonar microcítico 15.2
Melanoma 16.0
Carcinoma de células Merkel 17.0
Mesotelioma 18.0
Neoplasia mixto neuroendocrino no- neuroendocrino 19.0
Carcinoma neuroendocrino 20.0
Carcinoma neuroendocrino de células pequeñas 20.1
Tumor neuroendocrino 21.0
Osteosarcoma 22.0
Cáncer de ovario 23.0
Cáncer de páncreas 24.0
Paraganglioma 25.0
Cáncer de próstata 26.

In [6]:
# Importamos el fichero Excel con la información tanto de los genes como de su número correspondiente. 
# La ruta corresponde a la posición donde se encuentra almacenado el fichero Genes.xlsx
genes = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Datos/Genes.xlsx")
#print(genes)

# Creamos una variable que contenga solo los nombres de los genes.
mutaciones = genes["GEN"].unique()
#print(mutaciones)

#Creo un diccionario con los valores del fichero.
mutaciones_dic = dict(zip(genes["GEN"], genes["Número gen"]))
mutaciones_dic

# Usamos items() para recorrer los elementos del diccionario, devolviendo el gen con su respectivo valor.
# Es otra forma de recorrer el diccionario para obtener su contenido.
for gen, valor in mutaciones_dic.items():
    print(gen, valor)

ABL1 1
AKT1 2
AKT3 3
ALK 4
AR 5
AXL 6
BRAF 7
CCND1 8
CDK4 9
CDK6 10
CTNNB1 11
DDR2 12
EGFR 13
ERBB2 14
ERBB3 15
ERBB4 16
ERG 17
ESR1 18
ETV1 19
ETV4 20
ETV5 21
FGFR1 22
FGFR2 23
FGFR3 24
FGFR4 25
GNA11 26
GNAQ 27
HRAS 28
IDH1 29
IDH2 30
JAK1 31
JAK2 32
JAK3 33
KIT 34
KRAS 35
MAP2K1 36
MAP2K2 37
MET 38
MTOR 39
MYC 40
MYCN 41
NRAS 42
NTRK1 43
NTRK2 44
NTRK3 45
PDGFRA 46
PIK3CA 47
PPARG 48
RAF1 49
RET 50
ROS1 51
SMO 52


## 2.3. Definición de algunas variables. <a id = "2.3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [7]:
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes'

In [8]:
#Se inicializan las variables como listas vacías.
fecha_Data = []
NHC_Data = [] 
Nbiopsia_Data = [] 
texto_Data = []
#Los resultados de la función LeerFicherosPDF se almacenan en ficheros.
ficheros = LeerFicherosPDF(Ruta)

for ficheroPDF in ficheros:
    #os.path.normpath() normaliza la ruta resultante eliminando cualquier redundancia en la ruta y asegura que esté en un formato estandarizado.
    #os.path.join() une la ruta Ruta y el nombre del archivo ficheroPDF en una ruta completa. 
    lines = LeerDocumento(os.path.normpath(os.path.join(Ruta,ficheroPDF)))
    #print(lines)
    #Aplicamos la función BuscarValor para almacenar distintas palabras dentro de distintas variables
    NHC_Data.append(BuscarValor("NHC:", lines))
    Nbiopsia_Data.append(BuscarValor("biopsia:", lines))
    fecha_Data.append(BuscarValor("Fecha:", lines))
    texto_Data.append(BuscarValor("de la muestra:", lines))
    #GenerarImagen(Ruta, ficheroPDF)
print(NHC_Data)
print(Nbiopsia_Data)
print(fecha_Data)
print(texto_Data)

[['00021', '00021', '00021'], ['00022', '00022', '00022'], ['00023', '00023'], ['00024', '00024'], ['00025', '00025', '00025'], ['00026', '00026', '00026'], ['00027', '00027'], ['00028', '00028'], ['1234567', '1234567', '1234567'], ['1234567', '1234567']]
[['23B00021-A1', '23B00021-A1', '23B00021-A1'], ['22B00022-A2', '22B00022-A2', '22B00022-A2'], ['21B00023-A1', '21B00023-A1'], ['23B00024-A1', '23B00024-A1'], ['23C00025-A1', '23C00025-A1', '23C00025-A1'], ['23P00026', '23P00026', '23P00026'], ['22B00027-C4', '22B00027-C4'], ['23B00028-A2', '23B00028-A2'], ['23B000000-A1/CHIP100.1', '23B000000-A1/CHIP100.1', '23B000000-A1/CHIP100.1'], ['23B00000-A1/CHIP100.2', '23B00000-A1/CHIP100.2']]
[['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24

## 2.3.1. Diagnóstico. <a id = "2.3.1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [9]:
# Definimos las variables que vamos a usar. 
textoDiag = []
numeroDiag = []

# Recorremos los resultados alamacenados en la variable texto_Data para crear una variable sin repeticiones donde almacene los resultados. 
# Texto_Data almacena los resultados obtenidos en la funcion BuscarValor. Lo que queremos es crear una variable que almacene solo una vez cada uno de los resultados obtenidos en las listas. 
for i in texto_Data:
    sinduplicados = list(set(i))
    textoDiag.append([x for x in i if x in sinduplicados][0])
print(textoDiag)
    
# Teniendo importado ya el fichero de los diagnósticos, puedo usar este para recorrer la variable anterior textoDiag y en caso de que coincida
# con algún elemento del diccionario, nos devuelve su valor en una nueva variable llamada numeroDiag.
for diagnostico in textoDiag:
    valor = diagnosticos_dic.get(diagnostico)
    numeroDiag.append(valor)
    print(numeroDiag)


['Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Cáncer tiroideo', 'Cáncer gástrico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico']
[15.1]
[15.1, 15.1]
[15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0, 15.1, 15.1]


## 2.3.2 NHC. <a id = "2.3.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [11]:
# Definimos una nueva variable. 
NHC = []

#Recorremos cada uno de los elementos de NHC_Data para crear una variable nueva que almacene únicamente uno de los resultados de cada fichero.
for i in NHC_Data:
    sinduplicadosNHC = list(set(i))
    NHC.append([x for x in i if x in sinduplicadosNHC][0])
print(NHC)

['00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '1234567', '1234567']


## 2.3.3 Biopsia. <a id = "2.3.3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [12]:
lista_resultante =  []
elementos_vistos = set()

# Recorremos los valores de Nbiopsia_Data para crear una variable sin duplicados y así asegurarnos de que solo existe una copia del total de resultados obtenidos. 
for sublist in Nbiopsia_Data:
    sublist_sin_duplicados = []
    for elemento in sublist:
        if elemento not in elementos_vistos:
            sublist_sin_duplicados.append(elemento)
            elementos_vistos.add(elemento)
    lista_resultante.append(sublist_sin_duplicados)

print(lista_resultante)

NB_values = [elemento for sublist in lista_resultante for elemento in sublist]
print(NB_values)

[['23B00021-A1'], ['22B00022-A2'], ['21B00023-A1'], ['23B00024-A1'], ['23C00025-A1'], ['23P00026'], ['22B00027-C4'], ['23B00028-A2'], ['23B000000-A1/CHIP100.1'], ['23B00000-A1/CHIP100.2']]
['23B00021-A1', '22B00022-A2', '21B00023-A1', '23B00024-A1', '23C00025-A1', '23P00026', '22B00027-C4', '23B00028-A2', '23B000000-A1/CHIP100.1', '23B00000-A1/CHIP100.2']


In [13]:
# Nos interesa saber el valor del tercer elemento de cada uno de los valore de biopsia, ya que van a determinar sin son biopsia, citología o punción. 
biopsia = [x[2] for x in NB_values]
biopsia

['B', 'B', 'B', 'B', 'C', 'P', 'B', 'B', 'B', 'B']

## 2.3.4 Biopsia sólida. <a id = "2.3.4"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [14]:
# Para cada una de las posibles letras que pueden aparecer en esa posición, se le ha asignado un número
#B de biopsia = 1
#C de citología = 3
#P de punción = 2
B = "1"
C = "3"
P = "2"
Biopsia_solida = []

# Cada vez que encuentre una de las tres letras, almacenará su número correspondiente definido arriba en la nueva variable Biopsia_solida
for i in biopsia:
    if i == "B":
        Biopsia_solida.append(B)
        print("1")
    elif i == "P":
        Biopsia_solida.append(P)
        print("2")
    else:
        Biopsia_solida.append(C)
        print("3")
        
print(Biopsia_solida)

1
1
1
1
3
2
1
1
1
1
['1', '1', '1', '1', '3', '2', '1', '1', '1', '1']


## 2.3.5.Fecha. <a id = "2.3.5"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [15]:
# Como con fechas nos pasa lo mismo (que aparecen repetidas veces el mismo resultado y solo nos intersa una vez). Creamos una nueva variable para almacenar solo un resultado cada vez.
fechas = []
for i in fecha_Data:
    sinduplicados = list(set(i))
    fechas.append([x for x in i if x in sinduplicados][0])

print(fechas)

['24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '25-may-2023', '25-may-2023']


## 2.4. Definimos el resto de las variables. <a id = "2.4"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

## 2.4.1. Ensayos clínicos y tratamientos disponibles. <a id = "2.4.1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

A la hora de obtener el número de ensayos clínicos y tratamientos disponibles, lo hacemos de la misma forma. Como vemos, el número de ensayos/tratamientos se encuentra justo delante de la palabra. 

Aprovechamos ese método para crear un patrón que usando expresiones regulares, nos pueda devolver dicho valor. Para ello especificamos con r"(\d+)\s* que nos interesa la cadena de texto que contenga cualquier número de 0-9 ambos incluidos y que se encuentre justo antes de la palabra definida posteriormente (Ensayos clínicos y tratamientos disponibles). 
Tras determinar las variables de interés que vamos a usar, recorremos el texto de cada uno de los ficheros para buscar el patrón usando el método **re.search()**. Una vez encontrado, se convierte en entero (int()) para poder ser almacenado en la variable ensayos/tratamientos y usando group(1) para obtener solo el número. 
Añadimos con append los valores almacenados en ensayos/tratamientos a la variable definida arriba ensayos_finales/tratamientos_finales. 

Posteriormente se ha desarrollado una función que nos permite modificar los valores. En caso de no haber obtenido ningún número de ensayos/tratamientos, añadimos un 0 a la nueva variable. En caso de haber obtenido uno o más resultados, añadimos un 1. De esta forma estamos binarizando los resultados.

In [57]:
#Definimos el patrón de búsqueda
patron = r"(\d+)\s* Ensayos clínicos"
ficheros = LeerFicherosPDF(Ruta)
#También las variables como lsitas vacías
lista_ensayos = []
ensayos_finales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    
    ensayos = 0
    #Recorremos lines
    for line in lines:
        resultado = re.search(patron, line)
        #Cuando aparece el resultado, añadimos el entero a la variable 
        if resultado:
            ensayos = int(resultado.group(1))
            
    #Añadimos lista_ensayos a ensayos
    lista_ensayos.append(ensayos)   
    
    #print(ficheroPDF + " -> Ensayos: " + str(ensayos))
print(lista_ensayos) 

#Binarizamos la variable
for i in lista_ensayos:
    if i == 0:
        ensayos_finales.append(0)
    else:
        ensayos_finales.append(1)
        
print(ensayos_finales)

[6, 5, 0, 0, 3, 5, 1, 0, 6, 4]
[1, 1, 0, 0, 1, 1, 1, 0, 1, 1]


In [58]:
patron2 = r"(\d+)\s* Tratamientos disponibles"
ficheros = LeerFicherosPDF(Ruta)

lista_tratamientos = []
tratamientos_finales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.normpath(os.path.join(Ruta, ficheroPDF)))
    tratamientos = 0
    
    for line in lines:
        resultado = re.search(patron2, line)
        if resultado:
            tratamientos = int(resultado.group(1))
    
    lista_tratamientos.append(tratamientos)
    #print(ficheroPDF + " -> Tratamientos: " + str(tratamientos))
    
print(lista_tratamientos)

#Binarizamos la variable
for i in lista_tratamientos:
    if i == 0:
        tratamientos_finales.append(0)
    else:
        tratamientos_finales.append(1)
print(tratamientos_finales)

[6, 1, 0, 0, 3, 0, 0, 1, 6, 1]
[1, 1, 0, 0, 1, 0, 0, 1, 1, 1]


## 2.4.2. Número de chip y de paciente. <a id = "2.4.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [59]:
#Iteramos sobre cada fichero de la variable ficheros
for ficheroPDF in ficheros:
    #os.path.isfile() para verificar si la ruta corresponde a un archivo existente
    #os.path.join() y os.path.normpath() crean una ruta junto con el nombre del archivo ficheroPDF y que sea formato PDF
    if os.path.isfile(os.path.normpath(os.path.join(Ruta, ficheroPDF))) and ficheroPDF.endswith(".pdf"):
        print(ficheroPDF)

C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\1\Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf
C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\2\Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf
C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\3\Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf
C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\4\Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf
C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\5\Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf
C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\6\Sam

In [60]:
numero_paciente = [] 

for ficheroPDF in ficheros:
    if os.path.isfile(os.path.normpath(os.path.join(Ruta, ficheroPDF))) and ficheroPDF.endswith(".pdf"):
        #se separa la ruta del nombre del archivo siendo ruta1 la ruta y fichero1 el fichero
        ruta1, fichero1 = os.path.split(ficheroPDF)
        #Se divide el nombre del archivo en nombre y extensión. Con [0] obtenemos solo el nombre
        paciente = os.path.splitext(fichero1)[0]
        #Dentro del nombre, obtenemos el séptimo valor
        pacientes = paciente[7]
        #Agrega el número el paciente a la lista definida anteriormente
        numero_paciente.append(pacientes)
        print(numero_paciente)

['1']
['1', '2']
['1', '2', '3']
['1', '2', '3', '4']
['1', '2', '3', '4', '5']
['1', '2', '3', '4', '5', '6']
['1', '2', '3', '4', '5', '6', '7']
['1', '2', '3', '4', '5', '6', '7', '8']
['1', '2', '3', '4', '5', '6', '7', '8', '1']
['1', '2', '3', '4', '5', '6', '7', '8', '1', '2']


In [20]:
chip2 = []
for ficheroPDF in ficheros:
    if os.path.isfile(os.path.normpath(os.path.join(Ruta, ficheroPDF))) and ficheroPDF.endswith(".pdf"):
        # Esta expresión regular nos permite obtener el valor numérico (una o más cifras numéricas) siempre que estén entre v y _ (en este caso v100_)
        patron = r"v(\d+)_"
        resultado = re.search(patron, ficheroPDF)
        # Si aparece, almacenamos el valor en numero_chip para posteriormente añadir los valores a una nueva variable chip2.
        if resultado:
            numero_chip = resultado.group(1)
            chip2.append(numero_chip)
print(chip2)

['100', '100', '100', '100', '100', '100', '100', '100', '100', '100']


## 2.4.3. Mutaciones y derivados. <a id = "2.4.3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

## 2.4.3.1. Mutaciones totales. <a id = "2.4.3.1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [62]:
ficheros = LeerFicherosPDF(Ruta) 
max_mut = 0
genes_mut2 = {}
frecuencias_totales = []
patron_frecuencia = re.compile(r"\d{2}\.\d{2}")

for ficheroPDF in ficheros:
    nombreFichero = os.path.normpath(os.path.join(Ruta, ficheroPDF))
    lines = LeerDocumento(nombreFichero)
    total_mut = 0
    encontrados2 = []
    lista_frec = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            if mutacion == "FGFR4": 
                # Comprobamos que la siguiente posición a la del gen no sea p.(P136L). Si lo es pasamos, si no lo es añadimos un 1 a la variable total, añadimos la mutación a la variable encontrados 2 e imprimimos el valor.
                if (posicion < len(lines)-1 and lines[posicion + 1] == "p.(P136L)"):
                    pass
                else:
                    total_mut += 1
                    encontrados2.append(mutacion)
                    #print(ficheroPDF + " - Existe: " + mutacion)
        
            else:
                #Asumimos que la variable benigno es False usando boleanos. 
                benigno = False
                # Recorremos todas las posiciones
                for a in range(posicion+1, posicion+10):
                    #Si aparece Benign en las líneas, la variable definida anteriormente como False para a ser True.
                    if "Benign" in lines[a]:
                        benigno = True
                # Como las benignas no interesan, nos quedamos con las que siguen teniendo valor False
                if benigno == False:
                    # Añadimos 1 a la variable total_mut y añadimos la mutación a encontrados2. Imprimimos las mutaciones con sus respectivos ficheros
                    total_mut += 1
                    encontrados2.append(mutacion)
                    print(nombreFichero + " - Existe: " + mutacion)
                if not benigno:
                    for i in lines[posicion:posicion+10]:
                        #Usamos el patron definido anteriormente para obtener el porcentaje alélico de cada gen
                        resultado = re.search(patron_frecuencia, i)
                        if resultado:
                            frec = resultado.group()
                            lista_frec.append(frec)
                            
                    

    genes_mut2[ficheroPDF.replace("\\","_")] = encontrados2
    if total_mut > max_mut:
        max_mut = total_mut
        
    frecuencias_totales.append(lista_frec)
    
    #print(f"{ficheroPDF} - Número total de mutaciones : {total_mut}")
    #print(f"{ficheroPDF} - Frecuencias: {lista_frec}")


C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\1\Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\2\Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\4\Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: FGFR1
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\4\Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: PIK3CA
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\5\Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: CDK6
C:\Users\luciv\OneDrive

In [63]:
frecuencias_totales

[['49.56'],
 ['66.50'],
 [],
 [],
 ['21.75', '21.75'],
 ['27.51', '35.40'],
 ['46.17'],
 ['53.12', '60.64'],
 ['49.56'],
 ['66.50']]

In [64]:
# Creamos una nueva variable para almacenar solo los genes (valores) de la variable genes_mut2, ya que almacenaba tanto el nombre del fichero como los genes de interés que se modificaban.
mut = list(genes_mut2.values())
mut

[['MYC'],
 ['KRAS'],
 [],
 ['FGFR1', 'PIK3CA'],
 ['CDK6', 'MET'],
 ['CTNNB1', 'EGFR'],
 ['HRAS'],
 ['KIT', 'MYC'],
 ['MYC'],
 ['KRAS']]

In [65]:
# Creamos una nueva función para que almacene el número de mutaciones que hay en cada fichero, para ello aplicamos la función len sobre la variable mut.
num_mutaciones = [len(i) for i in mut]
print(num_mutaciones)

[1, 1, 0, 2, 2, 2, 1, 2, 1, 1]


In [66]:
# Como ya tenemos un diccionario con las mutaciones y sus números correspondientes, podemos usarlo. 
numero_iden = []
for i in mut:
    # Se busca en el diccionario el valor asociado al gen, devolviéndolo. En caso de no encontrar ninguno, se devuelve 0.
    valores = [mutaciones_dic.get(gen, 0) for gen in i]
    # Los números obtenidos se agregan a la variable numero_iden
    numero_iden.append(valores)

print(numero_iden)

[[40], [35], [], [22, 47], [10, 38], [11, 13], [28], [34, 40], [40], [35]]


In [67]:
fusiones = []
for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.normpath(os.path.join(Ruta,ficheroPDF)))
    variantes = []
    
    for linea in lines:
        for mutacion in mutaciones:
            # Nos interesa tener las fusiones de los genes. 
            # Definimos el patrón: cuaalquier letra (mayúsculas) o número que aparezca una o más veces seguida de - y cualquier valor que aparezca en la variable mutación.
            patronGen = re.compile(r"[A-Z0-9]{1,}-" + mutacion)
            # En caso de encontrar el patrón, se almacenan las fusiones en la variable resultados.
            resultado = re.search(patronGen, linea)
            if resultado:
                # En caso afirmativo, se crea una nueva expresión regular para obtener el ID de cada una de las fusiones.
                gen = resultado.group()
                patronVariante = re.compile(gen + "[.][A-Za-z0-9]{1,}[.][A-Za-z0-9]{1,}")
                resultadoVariante = re.search(patronVariante,linea)
                if resultadoVariante:
                    variante = resultadoVariante.group(0)
                    variantes.append(variante)
    fusiones.append(variantes)
    #print(ficheroPDF)
    #print(variantes)
                    
fusiones


[['EML4-ALK.E6aA20.AB374361', 'EML4-ALK.E6bA20.AB374362'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['EML4-ALK.E6aA20.AB374361', 'EML4-ALK.E6bA20.AB374362'],
 []]

## 2.4.3.2. Genes patogénicos. <a id = "2.4.3.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [68]:
for ficheroPDF in ficheros:
    nombreFichero = os.path.normpath(os.path.join(Ruta, ficheroPDF))
    lines = LeerDocumento(nombreFichero)
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)

            for a in range(posicion+1, min(posicion+10, len(lines))):
                # En caso de que aparezca la palabra Pathogeni en el archivo,imprime el fichero, la mutación y Pathogenic.
                if "Pathogeni" in lines[a]:
                    print(nombreFichero + " - Existe: " + mutacion + " - Pathogenic")
                

C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\2\Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS - Pathogenic
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\6\Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1 - Pathogenic
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\10\Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS - Pathogenic


In [69]:
patron_frecuencia = re.compile(r"\d{2}\.\d{2}")
frecuenciasPato = []

for ficheroPDF in ficheros:
    nombreFichero = os.path.normpath(os.path.join(Ruta, ficheroPDF))
    lines = LeerDocumento(nombreFichero)
    lista_frec = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    print(nombreFichero + "- Existe: " + mutacion + "- Pathogenic")
            
                    for i in lines[posicion:posicion+10]:
                        resultado = re.search(patron_frecuencia, i)
                        if resultado:
                            frec = resultado.group()
                            lista_frec.append(frec)
                            #frecuencias.append(frec)
    frecuenciasPato.append(lista_frec)
                            #print(f"   Frecuencia: {frec}")
        
    #print(f"{ficheroPDF} - Frecuencias: {lista_frec}")
    
frecuenciasPato


C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\2\Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf- Existe: KRAS- Pathogenic
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\6\Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf- Existe: CTNNB1- Pathogenic
C:\Users\luciv\OneDrive\Documentos\TFG_Lucia\GitHub\Automatizacion_PDF_scraping\INPUT\Informes\10\Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf- Existe: KRAS- Pathogenic


[[], ['66.50'], [], [], [], ['27.51'], [], [], [], ['66.50']]

In [70]:
patogen = {}

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.normpath(os.path.join(Ruta,ficheroPDF)))
    genpato2 = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    genpato2.append(mutacion)
                    #print(ficheroPDF + " - Existe: " + mutacion + " - Pathogenic")
            
                    
    
    patogen[ficheroPDF] = genpato2
    
patogen

{'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\\1\\Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf': [],
 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\\2\\Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf': ['KRAS'],
 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\\3\\Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf': [],
 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\\4\\Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf': [],
 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/INPUT/Informes\\5\\Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf': [],
 'C:/Users/luciv/OneDrive/Documentos/TFG_Luc

In [71]:
# Creamos una lista con los valores que son patogénicos.
patologicos = list(patogen.values())
patologicos

[[], ['KRAS'], [], [], [], ['CTNNB1'], [], [], [], ['KRAS']]

In [72]:
# Como en el caso anterior, a cada una de las mutaciones patogénicas le corresponde un valor numérico.
numero_iden_pato = []
for i in patologicos:
    valores = [mutaciones_dic.get(gen, 0) for gen in i]
    numero_iden_pato.append(valores)

numero_iden_pato

[[], [35], [], [], [], [11], [], [], [], [35]]

In [73]:
# Contamos el número de mutaciones patogénicas hay en cada fichero.
num_mutacionesPato = [len(i) for i in numero_iden_pato]
print(num_mutacionesPato)

[0, 1, 0, 0, 0, 1, 0, 0, 0, 1]


## 2.5. Variables de interés. <a id = "2.5"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [74]:
# Número de chip 
chip2

['100', '100', '100', '100', '100', '100', '100', '100', '100', '100']

In [75]:
# Número de paciente 
numero_paciente

['1', '2', '3', '4', '5', '6', '7', '8', '1', '2']

In [76]:
# NHC
NHC

['00021',
 '00022',
 '00023',
 '00024',
 '00025',
 '00026',
 '00027',
 '00028',
 '1234567',
 '1234567']

In [77]:
# Número de biopsia 
NB_values

['23B00021-A1',
 '22B00022-A2',
 '21B00023-A1',
 '23B00024-A1',
 '23C00025-A1',
 '23P00026',
 '22B00027-C4',
 '23B00028-A2',
 '23B000000-A1/CHIP100.1',
 '23B00000-A1/CHIP100.2']

In [78]:
# Biopsia sólida 
Biopsia_solida

['1', '1', '1', '1', '3', '2', '1', '1', '1', '1']

In [79]:
# Fecha de informe
print(fecha_Data)
print("---------------------------------------------------------------------------------------------------------------------")
print(fechas)

[['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['25-may-2023', '25-may-2023', '25-may-2023'], ['25-may-2023', '25-may-2023']]
---------------------------------------------------------------------------------------------------------------------
['24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '25-may-2023', '25-may-2023']


In [80]:
# Diagnóstico 
print(texto_Data)
print("------------------------------------------------------------------------------------------------------------------")
print(textoDiag)

[['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Cáncer tiroideo'], ['Cáncer gástrico'], ['Carcinoma pulmonar no microcítico', ''], ['Carcinoma pulmonar no microcítico', '']]
------------------------------------------------------------------------------------------------------------------
['Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Cáncer tiroideo', 'Cáncer gástrico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico']


In [81]:
# Número de diagnóstico 
numeroDiag

[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0, 15.1, 15.1]

In [82]:
# Total del número de mutaciones
num_mutaciones

[1, 1, 0, 2, 2, 2, 1, 2, 1, 1]

In [83]:
# Número de mutaciones patogénicas
num_mutacionesPato

[0, 1, 0, 0, 0, 1, 0, 0, 0, 1]

In [84]:
# Mutaciones detectadas totales 
mut

[['MYC'],
 ['KRAS'],
 [],
 ['FGFR1', 'PIK3CA'],
 ['CDK6', 'MET'],
 ['CTNNB1', 'EGFR'],
 ['HRAS'],
 ['KIT', 'MYC'],
 ['MYC'],
 ['KRAS']]

In [85]:
# Fusiones
fusiones

[['EML4-ALK.E6aA20.AB374361', 'EML4-ALK.E6bA20.AB374362'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['EML4-ALK.E6aA20.AB374361', 'EML4-ALK.E6bA20.AB374362'],
 []]

In [86]:
# % de frecuencia alélica
frecuencias_totales

[['49.56'],
 ['66.50'],
 [],
 [],
 ['21.75', '21.75'],
 ['27.51', '35.40'],
 ['46.17'],
 ['53.12', '60.64'],
 ['49.56'],
 ['66.50']]

In [87]:
# % frecuencias alélicas patológicas 
frecuenciasPato

[[], ['66.50'], [], [], [], ['27.51'], [], [], [], ['66.50']]

In [88]:
# Nº de la mutación específica detectada 
numero_iden

[[40], [35], [], [22, 47], [10, 38], [11, 13], [28], [34, 40], [40], [35]]

In [90]:
# Ensayos clínicos 
print(lista_ensayos)
print("-----------------------------------------------------------------------------------------------------------------")
print(ensayos_finales)

[6, 5, 0, 0, 3, 5, 1, 0, 6, 4]
-----------------------------------------------------------------------------------------------------------------
[1, 1, 0, 0, 1, 1, 1, 0, 1, 1]


In [91]:
# Fármaco aprobado 
print(lista_tratamientos)
print("-----------------------------------------------------------------------------------------------------------------")
print(tratamientos_finales)

[6, 1, 0, 0, 3, 0, 0, 1, 6, 1]
-----------------------------------------------------------------------------------------------------------------
[1, 1, 0, 0, 1, 0, 0, 1, 1, 1]


## 3. Creación de DataFrame. <a id = "3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

Join y merge son dos formas distintas de unir datos en Python usando la biblioteca pandas. 
Sin embargo, en este caso se ha usado JOIN porque nos garantiza que no se crearán filas ni columnas espurias al unir los PDF, esto se debe a que une los DF por índices en lugar de por columnas (este tipo de unión es característico de merge, por lo que es capaz de crear resultados espurios). 
En merge si se producen resultados espurios, esto ocurre cuando los datos tienen valores duplicados o vacíos o si los nombres de las columnas no coinciden en ambas tablas. Solo une los valores coincidentes de aquellas columnas con mismo nombre, eliminando el resto de las filas.

In [92]:
# PACIENTES
T1 = pd.DataFrame({'Número de chip': chip2, 'Número de paciente': numero_paciente, 'NHC': NHC, 
                    'Número de biopsia': NB_values, 'Biopsia sólida': Biopsia_solida, 'Fecha de informe': fechas})

In [93]:
# DIAGNOSTICO
T2 = pd.DataFrame({'Número de chip': chip2,'Número de biopsia': NB_values,'Diagnóstico': textoDiag, 
                   'Número del diagnóstico': numeroDiag})

In [94]:
# MUTACIONES
T3 = pd.DataFrame({'Número de chip': chip2, 'Número de biopsia': NB_values,'Mutaciones detectadas':mut, 
                   'Número de la mutación específica':numero_iden, 'Total del número de mutaciones': num_mutaciones, 
                   'Porcentaje de frecuencia alélica (ADN)': frecuencias_totales, 'Fusiones ID':fusiones})

In [95]:
# PATOGENICAS
T4 = pd.DataFrame({'Número de chip': chip2, 'Número de biopsia': NB_values,
                'Genes patogénicos': patologicos, 'Número de la mutación específica':numero_iden_pato, 
                '% frecuencia alélica':frecuenciasPato, 'Total de mutaciones patogénicas': num_mutacionesPato})

In [96]:
# INFORMACION
T5 = pd.DataFrame({'Número de chip': chip2, 'Número de biopsia': NB_values,'Ensayos clínicos': lista_ensayos,
                   'SI/NO ensayo':ensayos_finales, 'Fármaco aprobado': lista_tratamientos, 'SI/NO fármacos': tratamientos_finales})

In [97]:
tabla_unida = T1.join(T2.set_index(["Número de chip", "Número de biopsia"]), on=["Número de chip", "Número de biopsia"])
tabla_unida

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0
8,100,1,1234567,23B000000-A1/CHIP100.1,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1
9,100,2,1234567,23B00000-A1/CHIP100.2,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1


In [98]:
tabla_unida2 = tabla_unida.join(T3.set_index(["Número de chip", "Número de biopsia"]), on=["Número de chip", "Número de biopsia"])
tabla_unida2

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Mutaciones detectadas,Número de la mutación específica,Total del número de mutaciones,Porcentaje de frecuencia alélica (ADN),Fusiones ID
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[MYC],[40],1,[49.56],"[EML4-ALK.E6aA20.AB374361, EML4-ALK.E6bA20.AB3..."
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],1,[66.50],[]
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],0,[],[]
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[FGFR1, PIK3CA]","[22, 47]",2,[],[]
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CDK6, MET]","[10, 38]",2,"[21.75, 21.75]",[]
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CTNNB1, EGFR]","[11, 13]",2,"[27.51, 35.40]",[]
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[HRAS],[28],1,[46.17],[]
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,"[KIT, MYC]","[34, 40]",2,"[53.12, 60.64]",[]
8,100,1,1234567,23B000000-A1/CHIP100.1,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[MYC],[40],1,[49.56],"[EML4-ALK.E6aA20.AB374361, EML4-ALK.E6bA20.AB3..."
9,100,2,1234567,23B00000-A1/CHIP100.2,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],1,[66.50],[]


In [99]:
tabla_final = tabla_unida2.join(T5.set_index(["Número de chip", "Número de biopsia"]), on=["Número de chip", "Número de biopsia"])
tabla_final

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Mutaciones detectadas,Número de la mutación específica,Total del número de mutaciones,Porcentaje de frecuencia alélica (ADN),Fusiones ID,Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[MYC],[40],1,[49.56],"[EML4-ALK.E6aA20.AB374361, EML4-ALK.E6bA20.AB3...",6,1,6,1
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],1,[66.50],[],5,1,1,1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],0,[],[],0,0,0,0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[FGFR1, PIK3CA]","[22, 47]",2,[],[],0,0,0,0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CDK6, MET]","[10, 38]",2,"[21.75, 21.75]",[],3,1,3,1
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CTNNB1, EGFR]","[11, 13]",2,"[27.51, 35.40]",[],5,1,0,0
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[HRAS],[28],1,[46.17],[],1,1,0,0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,"[KIT, MYC]","[34, 40]",2,"[53.12, 60.64]",[],0,0,1,1
8,100,1,1234567,23B000000-A1/CHIP100.1,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[MYC],[40],1,[49.56],"[EML4-ALK.E6aA20.AB374361, EML4-ALK.E6bA20.AB3...",6,1,6,1
9,100,2,1234567,23B00000-A1/CHIP100.2,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],1,[66.50],[],4,1,1,1


In [100]:
tabla_unida3 = tabla_unida.join(T4.set_index(["Número de chip", "Número de biopsia"]), on=["Número de chip", "Número de biopsia"])
tabla_unida3

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Genes patogénicos,Número de la mutación específica,% frecuencia alélica,Total de mutaciones patogénicas
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],[66.50],1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[CTNNB1],[11],[27.51],1
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[],[],[],0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,[],[],[],0
8,100,1,1234567,23B000000-A1/CHIP100.1,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
9,100,2,1234567,23B00000-A1/CHIP100.2,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],[66.50],1


In [102]:
tabla_final_pato = tabla_unida3.join(T5.set_index(["Número de chip", "Número de biopsia"]), on=["Número de chip", "Número de biopsia"])
tabla_final_pato

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Genes patogénicos,Número de la mutación específica,% frecuencia alélica,Total de mutaciones patogénicas,Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,6,1,6,1
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],[66.50],1,5,1,1,1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,0,0,0,0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,0,0,0,0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,3,1,3,1
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[CTNNB1],[11],[27.51],1,5,1,0,0
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[],[],[],0,1,1,0,0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,[],[],[],0,0,0,1,1
8,100,1,1234567,23B000000-A1/CHIP100.1,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,6,1,6,1
9,100,2,1234567,23B00000-A1/CHIP100.2,1,25-may-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],[66.50],1,4,1,1,1


## 4. Exportación. <a id = "4"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [103]:
# Exportar el dataframe a un archivo Excel con índices
tabla_final_pato.to_excel('C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/Automatizacion_PDF_scraping/INPUT/Resultados/TablaPato.xlsx')

In [104]:
# Exportar el dataframe a un archivo Excel con índices
tabla_final.to_excel('C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/Automatizacion_PDF_scraping/INPUT/Resultados/TablaGeneral.xlsx')

In [105]:
# Divide el DataFrame en fragmentos de 80 líneas
fragmentos = np.array_split(tabla_final, len(tabla_final) // 80 + 1)
ruta2 = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/Automatizacion_PDF_scraping/INPUT/Resultados'

# Guarda cada fragmento en un archivo Excel separado
for i, fragmento in enumerate(fragmentos):
    nombre_archivo = f"tabla_final{i}.xlsx"
    ruta_archivo = os.path.join(ruta2, nombre_archivo)
    fragmento.to_excel(ruta_archivo, index=False)
    print(f"Archivo {nombre_archivo} guardado correctamente.")


Archivo tabla_final0.xlsx guardado correctamente.


In [106]:
# Divide el DataFrame en fragmentos de 80 líneas
fragmentos = np.array_split(tabla_final_pato, len(tabla_final_pato) // 80 + 1)
ruta2 = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/GitHub/Automatizacion_PDF_scraping/Automatizacion_PDF_scraping/INPUT/Resultados'

# Guarda cada fragmento en un archivo Excel separado
for i, fragmento in enumerate(fragmentos):
    nombre_archivo = f"patogenicos_{i}.xlsx"
    ruta_archivo = os.path.join(ruta2, nombre_archivo)
    fragmento.to_excel(ruta_archivo, index=False)
    print(f"Archivo {nombre_archivo} guardado correctamente.")


Archivo patogenicos_0.xlsx guardado correctamente.
